# ExoNAMD API v1.0


## Summary

The observer wants to compute the relative and/or absolute NAMD of:

- a given multiplanetary system;
- a subset of multiplanetary systems;
- all the known ones.

This tool handles all of the above cases.


In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import os
import numpy as np
import pandas as pd
import warnings

from exonamd.catalog import download_nasa_confirmed_planets
from exonamd.utils import ROOT
from exonamd.utils import fetch_aliases
from exonamd.utils import update_host
from exonamd.utils import update_planet
from exonamd.solve import solve_values
from exonamd.interp import interp_eccentricity
from exonamd.interp import interp_mass
from exonamd.interp import interp_inclination
from exonamd.interp import interp_sma


warnings.filterwarnings("ignore")
pd.options.display.max_columns = 20
pd.options.display.max_rows = 30
pd.options.mode.copy_on_write = True

### Task 1: getting the data

This task retrieves the parameters of confirmed systems from the NASA Exoplanet Archive database.


In [21]:
df, df_old = download_nasa_confirmed_planets(
    sy_pnum=1,
    from_scratch=False,
)

### Task 2: dealing with the aliases


Fetch aliases


In [22]:
aliases = fetch_aliases(df["hostname"].unique())

Fetched 3/3 entries on try 1


Curate aliases


In [18]:
df["hostname"] = df.apply(update_host, args=(aliases,), axis=1)
df["pl_name"] = df.apply(update_planet, args=(aliases,), axis=1)

In [9]:
# Double check that the names are consistent

for hostname in df["hostname"].unique():
    df_host = df[df["hostname"] == hostname]
    names = df_host["pl_name"]
    if len(set([name[:3] for name in names])) > 1:
        print(f"Inconsistent name for {hostname}")

### Task 3: computing missing values (if any) from simple equations


In [10]:
df[
    [
        "pl_orbsmax",
        "pl_ratdor",
        "st_rad",
        "pl_rade",
        "pl_ratror",
        "pl_orbper",
        "st_mass",
    ]
] = df.apply(solve_values, axis=1, result_type="expand")

### Task 4: storing the curated database


In [11]:
if df_old is not None:
    df_new = df.copy()
    df = pd.concat([df_new, df_old], ignore_index=True)
    df = df.drop_duplicates(keep="last")

df.to_csv(os.path.join(ROOT, "data", "exo.csv"), index=False)

In [12]:
# df = pd.read_csv(os.path.join(ROOT, "data", "exo.csv"))

Drop columns that are no longer needed


In [44]:
df.drop(
    columns=[
        "pl_ratdor",
        "st_rad",
        "pl_ratror",
        "pl_orbper",
        "st_mass",
    ],
    inplace=True,
)

### Task 5: input missing values (if any) by interpolation


Use nanmedian to thin down the data


In [45]:
keep_indices = []

for planet in df["pl_name"].unique():
    df_planet = df[df["pl_name"] == planet]
    default_index = df_planet[df_planet["default_flag"] == 1].index
    for col in [
        c
        for c in df_planet.columns
        if c not in ["hostname", "pl_name", "default_flag", "rowupdate"]
    ]:
        avg = np.nanmedian(df_planet[col].values)
        df.loc[default_index, col] = avg
    keep_indices.extend(default_index)

df.drop(df.index[~df.index.isin(keep_indices)], inplace=True)
df.drop(columns="default_flag", inplace=True)

Instantiate flags


In [46]:
df["flag"] = "0"

Solve missing eccentricity values


In [47]:
df[
    [
        "pl_orbeccen",
        "pl_orbeccenerr1",
        "pl_orbeccenerr2",
        "flag",
    ]
] = df.apply(interp_eccentricity, axis=1, result_type="expand")

Solve missing planetary mass values


In [48]:
df[
    [
        "pl_bmasse",
        "pl_bmasseerr1",
        "pl_bmasseerr2",
        "flag",
    ]
] = df.apply(interp_mass, axis=1, result_type="expand")

Drop columns that are no longer needed


In [49]:
df = df.drop(columns=["pl_rade", "pl_radeerr1", "pl_radeerr2"])

Remove systems where at least one planet has no mass or semi-major axis (if any)


In [50]:
remove_indices = []
for hostname in df["hostname"].unique():
    df_host = df[df["hostname"] == hostname]
    if df_host[["pl_bmasse", "pl_orbsmax"]].isnull().any().any():
        # print(
        #     f"Removing {hostname} "
        #     "due to no mass or semi-major axis."
        # )
        remove_indices.extend(df_host.index)
df.drop(df.index[df.index.isin(remove_indices)], inplace=True)

Solve missing values in inclinations


In [51]:
df[
    [
        "pl_orbincl",
        "pl_orbinclerr1",
        "pl_orbinclerr2",
        "flag",
    ]
] = df.apply(interp_inclination, args=(df,), axis=1, result_type="expand")

Solve missing values in semi-major axis uncertainties


In [52]:
df[
    [
        "pl_orbsmaxerr1",
        "pl_orbsmaxerr2",
        "flag",
    ]
] = df.apply(interp_sma, axis=1, result_type="expand")

### Task 6: storing the curated+interpolated database


In [53]:
df.to_csv(os.path.join(ROOT, "data", "exo_interp.csv"), index=False)